In [1]:
f_S(X) = sum(X.^2) ## S = sphere

f_S (generic function with 1 method)

In [6]:
XX = [1,2,3,4]
f_S(XX)

30

In [4]:
## set the hyperparameters
n = 2
Range = [-10,10] ## exploration range: Range^n
S = 10 ## number of bacteria
Sr = 4 ## number of bacteria removed in reproductive step
Nc = 20 ## chemotactic steps
Ns = 5 ## swim steps
Nre = 50 ## reproductive steps
Ned = 10 ## elimination and dispersal steps
Ped = 0.3 ## probability of elimination
Ci = (Range[2]-Range[1])/S; ## run-length unit

In [5]:
using LinearAlgebra ## "norm"
using StatsBase ## for function "sample"
## This function computes the classical (non-adaptive) BFO (Bacterial Foraging Optimization)
## Inputs:
## J = a function with domain R^n
## n = 2, dimension of the input of J
## Range = [-10,10], exploration range: Range^n
## S = 10, number of bacteria
## Sr = number of bacteria removed in reproductive step
## Nc = number of chemotactic steps
## Ns = number of swim steps
## Nre = number of reproductive steps
## Ned = elimination and dispersal steps
## Ped = probability of elimination
## Ci = (Range[2]-Range[1])/S; ## run-length unit
## Output: a dictionary that stores
## (1) the minimum value of J
## (2) the point achieving this minimu value
## (3) the path of each bacterium (for plotting illustration)
function BFO(J, Range, n = 2::Int, S = 10::Int, Sr = 4::Int, Nc = 20::Int, Ns = 5::Int, 
        Nre = 50::Int, Ned = 10::Int, Ped = 0.3::Float64, Ci = ((Range[2]-Range[1])/S)::Float64)
    ## randomly generate S bacteria in Range^n
    B_loc = (Range[2]-Range[1])*rand(n,S).+Range[1] ## B_loc = Bacteria locations
    ## a dictionary recording the path of bacterium i
    Path_Dict = Dict(i=>[zeros(n,0) B_loc[:,i]] for i=1:S)
    for l = 1:Ned ## index of elimination-dispersal steps
        for k = 1:Nre ## index of reproductive steps
            for j = 1:Nc ## index of chemotactic steps
                ## Chemotactic Step
                for i = 1:S ## index of bacterium
                    ## Tumble/Swim
                    Path_i = copy(B_loc[:,i]) ## record the path of bacterium i
                    m = 0 ## counter for swimming
                    delta_i = randn(n) ## random "tumble" direction (uniform on (n-1)-sphere)
                    while m<Ns
                        J_last = J(B_loc[:,i]) ## last fitness value
                        B_i_loc_new = B_loc[:,i] + Ci*delta_i/norm(delta_i) ## new location
                        J_new = J(B_i_loc_new) ## new fitness value
                        if J_new<J_last ## swim
                            m = m+1
                            J_last = copy(J_new)
                            B_loc[:,i] = B_i_loc_new
                            ## update the path of bacterium i
                            Path_i = [Path_i B_i_loc_new]
                        else
                            m = Ns ## don't swim 
                        end
                    end
                    ## update the path of bacterium i
                    if Path_Dict[i][:,end]!=B_loc[:,i]
                        Path_Dict[i] = [Path_Dict[i] B_loc[:,i]]
                    end
                end
            end
            if k<Nre
                ## Reproductive Step
                ## (!!!) I define the health of a bacterium as the J value of its current location
                Health = [J(B_loc[:,i]) for i=1:S]
                Health_sort = sortslices([Health collect(1:S)], dims = 1) ## sort the bacteria according to health
                B_survive = Array{Int,1}(Health_sort[1:S-Sr,2]) ## ## pick out the most healthy (S-Sr) bacteria
                B_rep = sort([B_survive;B_survive[1:Sr]]) ## reproduce the most healthy Sr bacteria
                B_loc = B_loc[:,B_rep]
                
                ## update the path dictionary
                for i = 1:S
                    if Path_Dict[i][:,end]!=B_loc[:,i]
                        Path_Dict[i] = [Path_Dict[i] B_loc[:,i]]
                    end
                end
            end
        end
        ## Elimination-Dispersal Step
        if l<Ned
            KillAlive = [sample([false, true], aweights([Ped,1-Ped])) for i=1:S] # true = alive, false = kill
            N_kill = S - sum(KillAlive) ## number of kiilled bacteria
            ## randomly generate a bacterium for each killed bacterium
            Alive = findall(KillAlive)
            Kill = setdiff(collect(1:S),Alive)
            B_loc = sortslices([[Alive'; B_loc[:,Alive]] [Kill';(Range[2]-Range[1])*rand(n,length(Kill)).+Range[1]]], dims = 2)[2:n+1,:]
            ## update the path dictionary
            for i = 1:S
                if Path_Dict[i][:,end]!=B_loc[:,i]
                    Path_Dict[i] = [Path_Dict[i] B_loc[:,i]]
                end
            end
        end
    end
    B_best = Int(sortslices([[J(B_loc[:,i]) for i=1:S] collect(1:S)], dims = 1)[1,2]) ## best bacterium
    X_best = B_loc[:,B_best] ## best location for minimizing J
    J_best = J(X_best) ## best (minimum) J value
    return Dict("Minimum"=>J_best, "Minimum Point"=>X_best, "Path_Dict"=>Path_Dict)
end

BFO (generic function with 10 methods)

In [ ]:
## Test the BFO function

In [3]:
J = f_S

ErrorException: invalid redefinition of constant J

In [6]:
R = [-100,100]
D = BFO(J,R)

Dict{String,Any} with 3 entries:
  "Path_Dict"     => Dict(7=>[40.544 66.7072 … 4.02024 -0.528119; 91.5179 61.26…
  "Minimum Point" => [-0.528119, 1.9503]
  "Minimum"       => 4.08258

In [7]:
D["Path_Dict"][1]

2×23 Array{Float64,2}:
 -81.2735  -79.1994   -57.8633  -30.6383  …   9.48297  -10.4507   -0.528119
  44.8392    4.89297  -28.9415   24.5262     -9.72234   -8.09522   1.9503  

In [16]:
## Plot the path of each bacterium
using Plots
plotly()
S = length(keys(D["Path_Dict"]))
scatter(xlim = R, ylim = R)
for i=1:S
    Path_i = D["Path_Dict"][i]
    if size(Path_i,2)!=1
        scatter!([Path_i[1,1]], [Path_i[2,1]], label = "Bacterium $i")
        plot!(Path_i[1,:], Path_i[2,:], label = "Bacterium $i")
    end
end
plot!()

## Modify/Imporve BFO

1. Allow "Range" as an n x 2 Array. The pth row is the range for the pth variable of J. 
2. Make the bacterial paths stay in Range.

Notations recap:
1. **J** = a function with domain R^n
2. **Range** = [-10,10], exploration range: Range^n <br><br>

3. **n** = 2, dimension of the input of J
4. **S** = 10, number of bacteria
5. **Sr** = 4,  number of bacteria removed in reproductive step
6. **Nc** = 20, number of chemotactic steps
7. **Ns** = 5, number of swim steps
8. **Nre** = 5, number of reproductive steps
9. **Ned** = 2, elimination and dispersal steps
10. **Ped** = 0.3, probability of elimination
11. **Ci** = (Range[2]-Range[1])/S, the run-length unit

In [63]:
J(X) = sqrt(sum(X));

In [64]:
using LinearAlgebra # "norm"
using StatsBase
Range = [0 2;0 3]
n = 2#::Int
S = 10#::Int, 
Sr = 4#::Int, 
Nc = 20#::Int, 
Ns = 5#::Int
Nre = 5#::Int
Ned = 2#::Int
Ped = 0.3#::Float64
Ci = ((norm(Range[:,2]-Range[:,1]))/(2*S));#::Float64    

In [62]:
    
    ## randomly generate S bacteria in Range
    Rand = rand(n,S) ## generate a random template
    B_loc = zeros(n,S)  ## B_loc = Bacteria locations
    for d = 1:n
        B_loc[d,:] = (Range[d,2]-Range[d,1])*Rand[d,:].+Range[d,1]
    end
    B_loc
    
    ## a dictionary recording the path of bacterium i
    Path_Dict = Dict(i=>[zeros(n,0) B_loc[:,i]] for i=1:S)
    for l = 1:Ned ## index of elimination-dispersal steps
        for k = 1:Nre ## index of reproductive steps
            for j = 1:Nc ## index of chemotactic steps
                ## Chemotactic Step
                println("Chemo-step $j")
                ToSwim = collect(1:S) ## monitor the swimming bacteria
                Tumble = RandUnit(n,S)
                m = 0 ## index of swimming
                while (ToSwim!=Array{Int64,1}())&(m<Ns) ## tumble/swim
                    m = m + 1
                    J_old = [J(B_loc[:,i]) for i in ToSwim]
    
                    B_loc_new = copy(B_loc)
                    B_loc_new[:,ToSwim] = B_loc[:,ToSwim].+Ci*Tumble[:,ToSwim]
                    ## Mirror back the out-of-range bacteria
                    for d = 1:n
                        ## mirror back bacteria tumbling out from below
                        B_loc_new[d,ToSwim] = (abs.(B_loc_new[d,ToSwim].-Range[d,1])).+(Range[d,1]) 
                        ## mirror back bacteria tumbling out from above
                        B_loc_new[d,ToSwim] = Range[d,2].-(abs.(Range[d,2].-B_loc_new[d,ToSwim]))
                    end
                    B_loc_new
    
                    ## Evaluate J at new locations
                    J_new = [J(B_loc_new[:,i]) for i in ToSwim]
    
                    ## Find out improved bacteria and update the swimming ones
                    ToSwim = ToSwim[findall(J_new.<J_old)]
    
                    for i in ToSwim
                        Path_Dict[i] = [Path_Dict[i] B_loc_new[:,i]]
                    end
                    B_loc[:,ToSwim] = B_loc_new[:,ToSwim]
                end ## end of tumble/swim
            end ## end of chemotactic steps
            
            if k<Nre
                ## Reproductive Step
                println("Repro-step $k")
                ## (!!!) I define the health of a bacterium as the J value of its current location
                Health = [J(B_loc[:,i]) for i=1:S]
                Health_sort = sortslices([Health collect(1:S)], dims = 1) ## sort the bacteria according to health
                B_survive = Array{Int,1}(Health_sort[1:S-Sr,2]) ## ## pick out the most healthy (S-Sr) bacteria
                B_rep = sort([B_survive;B_survive[1:Sr]]) ## reproduce the most healthy Sr bacteria
                B_loc = B_loc[:,B_rep]
                
                ## update the path dictionary
                for i = 1:S
                    if Path_Dict[i][:,end]!=B_loc[:,i]
                        Path_Dict[i] = [Path_Dict[i] B_loc[:,i]]
                    end
                end
            end
        end
        ## Elimination-Dispersal Step
        println("ED-step $l")
        if l<Ned
            KillAlive = [sample([false, true], aweights([Ped,1-Ped])) for i=1:S] # true = alive, false = kill
            N_kill = S - sum(KillAlive) ## number of kiilled bacteria
            ## randomly generate a bacterium for each killed bacterium
            Alive = findall(KillAlive)
            Kill = setdiff(collect(1:S),Alive)
            ## Rebuild the killed bacteria in random locations in Range
            ED = [Alive'; B_loc[:,Alive]]
            for dead in Kill
                Revive_dead = [dead;[(Range[d,2]-Range[d,1])*rand(1)[1]+Range[d,1] for d=1:n]]
                ED = [ED Revive_dead]
            end
            B_loc = sortslices(ED, dims = 2)[2:n+1,:]
            ## update the path dictionary
            for i = 1:S
                if (Path_Dict[i][:,end])!=B_loc[:,i]
                    Path_Dict[i] = [Path_Dict[i] B_loc[:,i]]
                end
            end
        end
    end
    B_best = Int(sortslices([[J(B_loc[:,i]) for i=1:S] collect(1:S)], dims = 1)[1,2]) ## best bacterium
    X_best = B_loc[:,B_best] ## best location for minimizing J
    J_best = J(X_best) ## best (minimum) J value
    Dict("Minimum"=>J_best, "Minimum Point"=>X_best, "Path_Dict"=>Path_Dict)


Chemo-step 1
Chemo-step 2
Chemo-step 3
Chemo-step 4
Chemo-step 5
Chemo-step 6
Chemo-step 7
Chemo-step 8
Chemo-step 9
Chemo-step 10
Chemo-step 11
Chemo-step 12
Chemo-step 13
Chemo-step 14
Chemo-step 15
Chemo-step 16
Chemo-step 17
Chemo-step 18
Chemo-step 19
Chemo-step 20
Repro-step 1
Chemo-step 1
Chemo-step 2
Chemo-step 3
Chemo-step 4
Chemo-step 5
Chemo-step 6
Chemo-step 7
Chemo-step 8
Chemo-step 9
Chemo-step 10
Chemo-step 11
Chemo-step 12
Chemo-step 13
Chemo-step 14
Chemo-step 15
Chemo-step 16
Chemo-step 17
Chemo-step 18
Chemo-step 19
Chemo-step 20
Repro-step 2
Chemo-step 1
Chemo-step 2
Chemo-step 3
Chemo-step 4
Chemo-step 5
Chemo-step 6
Chemo-step 7
Chemo-step 8
Chemo-step 9
Chemo-step 10
Chemo-step 11
Chemo-step 12
Chemo-step 13
Chemo-step 14
Chemo-step 15
Chemo-step 16
Chemo-step 17
Chemo-step 18
Chemo-step 19
Chemo-step 20
Repro-step 3
Chemo-step 1
Chemo-step 2
Chemo-step 3
Chemo-step 4
Chemo-step 5
Chemo-step 6
Chemo-step 7
Chemo-step 8
Chemo-step 9
Chemo-step 10
Chemo-step 11
Che

Dict{String,Any} with 3 entries:
  "Path_Dict"     => Dict(7=>[1.59505 1.4381 … 0.0535217 0.00548918; 0.640932 0…
  "Minimum Point" => [0.00548918, 0.0867604]
  "Minimum"       => 0.303726

In [60]:
[1;[(Range[d,2]-Range[d,1])*rand(1)[1]+Range[d,1] for d=1:n]]

3-element Array{Float64,1}:
 1.0               
 1.637697225833576 
 1.9784196189625658